In [ ]:
import syft_process_manager as syftpm

In [ ]:
# To reset notebook:
# syftpm.ProcessManager().remove_all()

syftpm.list()

## Running a process with syft-process-manager

syft-process-manager can run processes in two ways:

`syftpm.run` is a low-level interface to run a command `list[str]` as a process. It will inherit the current environment, and we can add any additional env if we want.

For example, we can run:

In [ ]:
import sys
import syft_process_manager as syftpm

code_to_execute = """
import time
import os

my_env_var = os.environ["MY_ENV_VAR"]
for i in range(10):
  print(i, my_env_var, flush=True)
  time.sleep(1)
"""

# NOTE: we use sys.executable to ensure we're using the same python env
handle = syftpm.run(
    name="simple-printer",
    env={"MY_ENV_VAR": "Hello World!"},
    cmd=[
        sys.executable,
        "-c",
        code_to_execute
      ]
)

In [ ]:
# Print info and logs
print(handle)
print()
print("stdout logs:")
print(handle.stdout.tail(10))

In [ ]:
handle.terminate()
print("status:", handle.status)

## Running a python function

Additionally, `syftpm.run_function` can run a python function instead of a command. Internally, it will use exactly the same functions under the hood as `syftpm.run`, and adds some additional features:

- It **pickles** your function with [cloudpickle](https://github.com/cloudpipe/cloudpickle) to execute it in a different process
- It sets up a simple **logger** with configurable `log_level`, that prints to stdout
- It sets up a periodic **health check**, that writes to `handle.config.health_file` every 5 seconds
- It has an optional **TTL (Time To Live)**, which limits the lifetime of the process, and automatically kills the process if it runs for longer.

Example usage:

In [ ]:
import time

def print_forever(msg: str) -> None:
    i = 0
    while True:
        print(f"{i}: {msg}", flush=True)
        i += 1
        time.sleep(1)


print_forever_handle = syftpm.run_function(
    print_forever,  # Run print_forever
    "Hello World!", # with this `msg`
    name="print-forever",
    ttl_seconds=3600, # run for 1 hour
    log_level="DEBUG",
    overwrite=True, # Stop and remove print-forever if it already exists
)

In [ ]:
print_forever_handle

## Using the ProcessManager

Under the hood, all top-level functions use the `ProcessManager`. This class has a few more functions to manage multiple processes at the same time.

Note that `ProcessManager` is fully stateless, so there is no need to pass around references of the process manager.

In [ ]:
pm = syftpm.ProcessManager()

# Show all processes registered with this process manager. This includes processes that are currently not running.
pm.list()

In [ ]:
# Get a process by name
simple_printer = pm.get("simple-printer")
print(simple_printer)

In [ ]:
# terminate_all terminates all processes, but the handles will still exist and can be re-started
pm.terminate_all()

pm.list() # all processes are stopped

In [ ]:
# remove_all terminates all processes, and removes the config and runtime artifacts in process_dir
pm.remove_all()

pm.list() # all processes are removed

## Persistence and internals

All state of the process is persisted on disk in a single folder. Let's start a new process and investigate what this looks like

In [ ]:
print_forever_handle = syftpm.run_function(
    print_forever,
    "Hello World!",
    name="print-forever",
    ttl_seconds=3600,
    log_level="DEBUG",
    overwrite=True,
)

In [ ]:
import time
time.sleep(1) # Wait for first health check

process_dir = print_forever_handle.config.process_dir
!ls {process_dir}

The files config, health, process_state, stderr, and stdout are all directly implemented on the handle (e.g. `handle.health` directly reads `health.json`).
Out of these variables, everything is always read from disk without caching, except for the static `handle.config`.

function.pkl is specific to the `syftpm.run_function` API, it contains our pickled function

In [ ]:
# Config holds all static process configuration:
print(print_forever_handle.config.model_dump_json(indent=2))

In [ ]:
# Process state holds the runtime information, and will change when the process is restarted
print(print_forever_handle.process_state.model_dump_json(indent=2))

In [ ]:
# Health is an optional file implemented by the process, the process can write periodic health checks to this file
print(print_forever_handle.health.model_dump_json(indent=2))

In [ ]:
# stdout and stderr both have a LogStream class to easily interface with the log files. For example:
print(print_forever_handle.stdout.tail(10))

In [ ]:
print_forever_handle.terminate()
print_forever_handle.status